In [5]:
import pandas as pd
import json

In [6]:
def save_fileInJSON(content ,strRoot : str, mode = 'w'):
    with open(strRoot,mode) as file:
        file.write(json.dumps(content))

In [7]:
def get_compressedData(listContent_One : list, listContent_Two : list, listHead : list):
    listVar = []
    for i in range(min(len(listContent_One), len(listContent_Two))):
        listVar.append( {listHead[0]:listContent_One[i], listHead[1]:listContent_Two[i]} )
    return listVar

In [8]:
data = pd.read_csv(r'C:\Users\major\Documents\GitHub\MoskowCityHack\RTG_Python\Scrap\data_vk.csv')

In [9]:
data = data.rename(columns={"Название площадки": "name", "Ссылка на сообщество": "link"
                    , "Количество участников": "subs", "Охват рекламы": "oxvat_rek"
                    , "Охват по подписчикам": "oxvat_subs", "Стоимость": "cost",
                     "Длительность размещения": "late_time"})

In [10]:
data =data.drop(['link','oxvat_rek','late_time'],axis=1)

In [11]:
data['predictCTR'] = (0.89/100)

In [12]:
data['CPC_predict'] = data['cost']/(data['predictCTR']*data['oxvat_subs'])

In [13]:
average_average_bank = 4000
ratio_effectiv = 0.05

In [14]:
data['predict_match_views'] = (data['oxvat_subs']*data['predictCTR']).astype(int)

In [15]:
data = data.sort_values(by=['predict_match_views'],ascending=False)

In [16]:
data['predict_match_users'] = (data['predict_match_views']*ratio_effectiv).astype(int)

In [17]:
data['predict_profit_bank'] = data['predict_match_users']*average_average_bank

In [18]:
data = data[data['predict_match_users']>0]

In [19]:
data['cost_one_user'] = (data['cost']/data['predict_match_users']).astype(int)

In [20]:
data['predict_profit_bank_prize'] = data['predict_profit_bank'] - data['cost']

In [21]:
data = data.sort_values(by=['predict_profit_bank_prize'],ascending=False)

In [22]:
data.index = data.name

Профит банка

In [23]:
save_fileInJSON(get_compressedData(data.predict_profit_bank_prize.index.tolist() ,  data.predict_profit_bank_prize.values.tolist(), ['PlatformName','PredictProfit']), 'JSON_RTG/bankProfit_VK.json')

Стоимость одного клиента

In [24]:
save_fileInJSON(get_compressedData(data.cost_one_user.index.tolist() ,  data.cost_one_user.values.tolist(), ['CountOfClient','PlatformName']), 'JSON_RTG/PricePerClient_VK.json')

Количество новых клиентов

In [25]:
save_fileInJSON(get_compressedData(data.predict_match_users.index.tolist() ,  data.predict_match_users.values.tolist(), ['CountOfClient','PlatformName']), 'JSON_RTG/CountOfNewClients_VK.json')